In [270]:
#Import the necessary packages and donwload treebank
import nltk
import numpy as np
from nltk.corpus import treebank
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import re
import random
from nltk.corpus import movie_reviews
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

nltk.download('treebank')

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\VENUS\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!


True

In [271]:
# Load the Treebank corpus for training
treebank_sents = treebank.tagged_sents()

# Printing a sample
treebank_sents[0]

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.')]

In [272]:
"""I have chosen four tags for ease:
N:Noun
V:Verb
ADJ:Adjective
ADV:Adverb
PRON:Pronoun
ART:Determinant
"""
#map the detailed tag mappings of treebank to the above 6 rest all are tagged as OTH
tag_mapping = {
    'NN': 'N', 'NNS': 'N', 'NNP': 'N', 'NNPS': 'N',  # Nouns
    'VB': 'V', 'VBD': 'V', 'VBG': 'V', 'VBN': 'V', 'VBP': 'V', 'VBZ': 'V',  # Verbs
    'JJ': 'ADJ', 'JJR': 'ADJ', 'JJS': 'ADJ',  # Adjectives
    'RB': 'ADV', 'RBR': 'ADV', 'RBS': 'ADV',  # Adverbs
    'PRP': 'PRON', 'PRP$': 'PRON', 'WP': 'PRON', 'WP$': 'PRON',  # Pronouns
    'DT': 'ART',  # Determinants
    'IN': 'PREP'  # Prepositions
}


In [273]:
#Create a new dataset considering only 6 tags
simplified_tagged_sentences = []
# Iterate through the original Treebank tagged sentences
for sentence in treebank.tagged_sents():
    simplified_sentence = [(word, tag_mapping.get(tag, 'OTH')) for word, tag in sentence]
    simplified_tagged_sentences.append(simplified_sentence)
#Print sample:
simplified_tagged_sentences[0]

[('Pierre', 'N'),
 ('Vinken', 'N'),
 (',', 'OTH'),
 ('61', 'OTH'),
 ('years', 'N'),
 ('old', 'ADJ'),
 (',', 'OTH'),
 ('will', 'OTH'),
 ('join', 'V'),
 ('the', 'ART'),
 ('board', 'N'),
 ('as', 'PREP'),
 ('a', 'ART'),
 ('nonexecutive', 'ADJ'),
 ('director', 'N'),
 ('Nov.', 'N'),
 ('29', 'OTH'),
 ('.', 'OTH')]

In [274]:
#These are our tags
simplified_tags = ['START', 'N', 'V', 'ADJ', 'ADV','PRON','ART','PREP','OTH', 'END']

In [275]:
# Define a dictionary
 
tag_to_index={'START': 0, 'N': 1, 'V': 2, 'ADJ': 3, 'ADV': 4,'PRON':5,'ART':6,'PREP':7,'OTH': 8, 'END': 9}

In [276]:
#Calculate tag transition probablities
num_tags=len(simplified_tags)
tag_transition_matrix=np.zeros((num_tags,num_tags),dtype=float)
for sentence in simplified_tagged_sentences:
    tags = [tag for (_, tag) in sentence]
    
    # Include transitions from START to the first tag and from the last tag to END
    tags = ['START'] + tags + ['END']
    for i in range(len(tags)-1):
        cur_tag=tag_to_index[tags[i]]
        next_tag=tag_to_index[tags[i+1]]
        tag_transition_matrix[cur_tag][next_tag]+=1 #increment the count in tag transition matrice
#Normalise for GETTING PROB
row_sums=tag_transition_matrix.sum(axis=1)
tag_transition_matrix/=row_sums[:,np.newaxis]

C:\Users\VENUS\AppData\Local\Temp\ipykernel_37872\3570715226.py:15: RuntimeWarning: invalid value encountered in divide
  tag_transition_matrix/=row_sums[:,np.newaxis]


In [277]:
#View the tag TRANSITION MATRIX of our dataset (7*7 2 extra for START and END tags)
tag_transition_matrix

array([[0.00000000e+00, 2.91517629e-01, 9.45324476e-03, 4.11343894e-02,
        4.59887583e-02, 7.35820133e-02, 2.31221257e-01, 1.29024016e-01,
        1.78078692e-01, 0.00000000e+00],
       [0.00000000e+00, 2.64073163e-01, 1.31014653e-01, 1.21245713e-02,
        1.54501680e-02, 4.67662036e-03, 5.95836076e-03, 1.76672325e-01,
        3.89337306e-01, 6.92832646e-04],
       [0.00000000e+00, 1.18303395e-01, 1.21389570e-01, 6.97950463e-02,
        7.42264778e-02, 3.80628314e-02, 1.42755401e-01, 9.80454222e-02,
        3.37421856e-01, 0.00000000e+00],
       [0.00000000e+00, 6.99390339e-01, 1.18805690e-02, 6.64373925e-02,
        3.75175864e-03, 6.25293106e-04, 4.84602157e-03, 7.76926684e-02,
        1.35375957e-01, 0.00000000e+00],
       [0.00000000e+00, 2.13832275e-02, 3.51152690e-01, 1.31306382e-01,
        8.05212162e-02, 5.67991981e-03, 5.11192783e-02, 1.24290010e-01,
        2.34213164e-01, 3.34112930e-04],
       [0.00000000e+00, 2.09353307e-01, 4.08111071e-01, 7.30727073e-02,
   

In [278]:
#Calculate the emission probablities
# Define a dictionary to store emission counts
emission_counts = {tag: {} for tag in simplified_tags}

# Count occurrences
for sentence in simplified_tagged_sentences:
    for word, tag in sentence:
        #if first time encountered then initialise to 1
        if word not in emission_counts[tag]:
            emission_counts[tag][word] = 1
        #increment by 1
        else:
            emission_counts[tag][word] += 1
# Convert counts to probabilities by dividing by the total occurrences of each tag
tag_counts = {tag: sum(emission_counts[tag].values()) for tag in simplified_tags}
emission_probabilities = {tag: {word: count/tag_counts[tag] for word, count in emission_counts[tag].items()} for tag in simplified_tags}
#View how emission probabilities dictionary look
emission_probabilities['PRON']

{'it': 0.17391304347826086,
 'its': 0.12130069419071976,
 'We': 0.017172086225794667,
 'our': 0.009499451954694921,
 'who': 0.059554256485202774,
 'us': 0.009499451954694921,
 'he': 0.08403361344537816,
 'you': 0.02557544757033248,
 'It': 0.03653635367190355,
 'they': 0.07672634271099744,
 'He': 0.026671538180489587,
 'their': 0.06613080014614542,
 'them': 0.02557544757033248,
 'What': 0.006941907197661673,
 'what': 0.017902813299232736,
 'we': 0.0175374497625137,
 'They': 0.01936426744610888,
 'his': 0.04420898794300329,
 'whose': 0.005115089514066497,
 'him': 0.005845816587504567,
 'You': 0.00876872488125685,
 'she': 0.028132992327365727,
 'She': 0.0076726342710997444,
 'themselves': 0.004384362440628425,
 'itself': 0.0025575447570332483,
 'I': 0.041286079649251003,
 'Its': 0.00401899890390939,
 'her': 0.018633540372670808,
 'Their': 0.0010960906101571063,
 'whom': 0.0021921812203142127,
 'one': 0.0007307270734380709,
 'His': 0.001826817683595177,
 'Her': 0.0014614541468761417,
 'you

In [279]:
#Just checking the sum of the probabilities hould be almost 1
print(sum(emission_probabilities['N'].values()))
print(sum(emission_probabilities['V'].values()))
print(sum(emission_probabilities['ADJ'].values()))
print(sum(emission_probabilities['ADV'].values()))

0.9999999999999325
0.9999999999999488
1.0000000000000384
0.999999999999995


In [280]:
simplified_tags

['START', 'N', 'V', 'ADJ', 'ADV', 'PRON', 'ART', 'PREP', 'OTH', 'END']

In [281]:
def viterbi_tagger(sentence):
    
    sentence = sentence.split(' ')
    # Number of tags
    n = len(sentence)
    m = len(simplified_tags)
    
    # This dp array will contain the probabilities, with dimensions (n, m)
    dp = np.zeros((n, m))
    epsilon = 0.00001 # a very small number to handle out-of-vocabulary words
    
    # This array will store the path to the current state
    parent = np.zeros((n, m), dtype=int)
    
    # Let's initialize the first row
    for j, tag in enumerate(simplified_tags): # go over the tags
        if tag == 'START' or tag=='END': # if tag is 'START', continue
            continue
        # get the emission probabilities for the first word
        emission = emission_probabilities[tag].get(sentence[0], epsilon)
        dp[0][j] = tag_transition_matrix[0][j] * emission
#     print(dp)    
    # fill the matrix for the rest of the values
    for i in range(1, n):
        for j, tag in enumerate(simplified_tags):
            if tag == 'START' or tag=='END': # if tag is 'START', continue
                continue
            max_prob = 0
            best_prev_tag = -1
            # This loop goes over each tag of the previous word
            # and calculates the maximum probability as well as the argmax
            for k, prev_tag in enumerate(simplified_tags):
                if tag == 'START' or tag=='END': # if tag is 'START', continue
                    continue
                prob = dp[i-1][k] * tag_transition_matrix[k][j]
                if prob > max_prob:
                    max_prob = prob
                    best_prev_tag = k
            emission = emission_probabilities[tag].get(sentence[i], epsilon)
            dp[i][j] = max_prob * emission
            parent[i][j] = best_prev_tag
#     print(dp)
    best_path = []
    # Start from the best last tag and k
    best_last_tag = np.argmax(dp[-1])
    best_path.append(best_last_tag)
    
    for i in range(n-1, 0, -1):
        best_last_tag = parent[i][best_last_tag]
        best_path.append(best_last_tag)
    
    best_path = best_path[::-1]
    return [simplified_tags[tag_idx] for tag_idx in best_path]


In [282]:
viterbi_tagger('Ram is a nice boy')

['N', 'V', 'ART', 'ADJ', 'N']

In [283]:
#Examples

In [284]:
print(viterbi_tagger('Pierre is a nice boy'))
print(viterbi_tagger('I like the NLP course'))
print(viterbi_tagger('He is the director of the company'))

['N', 'V', 'ART', 'ADJ', 'N']
['PRON', 'V', 'ART', 'N', 'N']
['PRON', 'V', 'ART', 'N', 'PREP', 'ART', 'N']


In [285]:

# Load movie reviews
documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

# Shuffle the documents and split into data and labels
 
random.shuffle(documents)
texts = [" ".join(words) for words, _ in documents]
labels = [cat for _, cat in documents]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(texts, labels, test_size=0.2,random_state=1)

def load_glove_embeddings(path):
    embeddings = {}
    with open(path, 'r',encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

glove_embeddings = load_glove_embeddings('glove.6B.200d.txt')

def text_to_embedding(text, embeddings):
    words = text.split()
    vectors = [embeddings[word] for word in words if word in embeddings]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(200)  # Assuming you are using 200d GloVe embeddings

X_train_embedded = np.array([text_to_embedding(text, glove_embeddings) for text in X_train])
X_test_embedded = np.array([text_to_embedding(text, glove_embeddings) for text in X_test])



clf = RandomForestClassifier()
clf.fit(X_train_embedded, y_train)

y_test_pred = clf.predict(X_test_embedded)


In [286]:
print(classification_report(y_test, y_test_pred))

              precision    recall  f1-score   support

         neg       0.64      0.71      0.67       193
         pos       0.70      0.63      0.66       207

    accuracy                           0.67       400
   macro avg       0.67      0.67      0.67       400
weighted avg       0.67      0.67      0.67       400



In [287]:


# POS Tagging using Viterbi
def tag_fetch(sentence):
    return viterbi_tagger(sentence)

#tag weights We assign weights only to get the adjectives and adverbs
#as they majorly decribe the sentiment
tag_weights = {
    'N': 0, 'V': 0, 'ADJ': 1.0, 'ADV': 1.0,
    'PRON':0, 'ART':0, 'PREP':0.0, 'OTH':0.0
}

# Dividing the Text into Sections of 30 words each
def section_text(data,section_size=30):
    items = re.split(r'[.\t ]+', data)
    sections = [' '.join(items[i:i+section_size]) for i in range(0, len(items), section_size)]
    return sections

# Convert Text to Weighted Vector
#This is the crux here we multiply the adjective and adverb embeddings by 1 and rest by zero
def text_to_vector(data, embedding_model, tag_weights):
    items = data.split()
    tags = tag_fetch(data)
    vectors_list = []
    
    for item, tag in zip(items, tags):
        wt = tag_weights.get(tag, 1.0)
        if item in embedding_model:
            vectors_list.append(embedding_model[item] * wt)
        else:
            vectors_list.append(np.zeros(200))

    if vectors_list:
        return np.mean(vectors_list, axis=0)
    else:
        return np.zeros(200)

# Vector Generation for Sections
#Take the mean of all Glove embeddings of all the words as the representation of the text
def section_to_vector(data, embedding_model, tag_weights,section_size=30):
    sections = section_text(data, section_size)
    section_vectors = [text_to_vector(section, embedding_model, tag_weights) for section in sections]
    return np.mean(section_vectors, axis=0)

# Load Embedding Model
#Create a dictionry of word vs Glove Embeddings
def fetch_embeddings(filepath):
    embedding_dict = {}
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vec = np.asarray(values[1:], dtype='float32')
            embedding_dict[word] = vec
    return embedding_dict

embeddings_dict = fetch_embeddings('glove.6B.200d.txt')



In [288]:
# Convert All Data to Weighted Vectors
X_train_vec_vt = np.array([section_to_vector(text, embeddings_dict, tag_weights) for text in X_train])
X_test_vec_vt = np.array([section_to_vector(text, embeddings_dict, tag_weights) for text in X_test])

# Train and Evaluate the Model
model = RandomForestClassifier()
model.fit(X_train_vec_vt, y_train)
predicted_labels_viterbi = model.predict(X_test_vec_vt)

model_accuracy = accuracy_score(y_test, predicted_labels_viterbi)
print(f"Model Accuracy: {model_accuracy * 100:.2f}%")


Model Accuracy: 69.50%


In [289]:
print(classification_report(y_test,predicted_labels_viterbi))

              precision    recall  f1-score   support

         neg       0.68      0.70      0.69       193
         pos       0.71      0.69      0.70       207

    accuracy                           0.69       400
   macro avg       0.70      0.70      0.69       400
weighted avg       0.70      0.69      0.70       400

